In [1]:
import os
os.chdir('/mnt/ialabnas/homes/fidelrio/systematic-text-representations/')

import json
from pathlib import Path
import random
import pprint

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


import torch

from config import load_config
from data import build_datasets, build_loader, build_detailed_test_dataloaders
from data import CollatorForMaskedSelectedTokens, CollatorForMaskedRandomSelectedTokens, IdentityCollator
from data import ALL_POSSIBLE_COLORS
from model import MultimodalModel, MultimodalPretrainingModel
from utils import load_checkpoint
from lightning import Trainer, seed_everything
from tqdm.auto import tqdm, trange

from PIL import Image

from torch.utils.data import DataLoader, Subset, ConcatDataset
from torch.nn.functional import softmax

import numpy as np
from sklearn.decomposition import PCA

pp = pprint.PrettyPrinter(indent=2)

2024-09-17 15:18:41.933016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-8.0/lib64/
2024-09-17 15:18:41.933044: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def scene_tensor_to_txt(tensor):
    return ' '.join([processor.inv_vocabulary[t] for t in tensor.tolist()])

def print_scene_tensor(tensor):
    scene_text = scene_tensor_to_txt(tensor)
    print(scene_text.replace('[PAD]', '').replace('[SEP]','\n     '))
    
def print_parallel(tensor0, tensor1, tensor2, confidences, titles):
    ttl0, ttl1, ttl2 = titles
    print(f'{ttl0:6.6s} {ttl1:6.6s} {ttl2:6.6s}')
    for t0, t1, t2, conf in zip(
            tensor0.tolist(), tensor1.tolist(), tensor2.tolist(), confidences.tolist()):
        w0 = processor.inv_vocabulary[t0]
        w1 = processor.inv_vocabulary[t1]
        w2 = processor.inv_vocabulary[t2]
        
        if w0 == '[SEP]':
            print()
            continue
        if w0 == '[PAD]':
            break
        
        print_txt = f'{w0:6.6s} {w1:6.6s} {w2:6.6s} ({conf:.4f})'
        if w0 != w2:
            print_txt = bold(print_txt)        

        print(print_txt)

def bold(text):
    return ("\033[1m" + text + "\033[0m")

In [3]:
device = torch.device('cuda')

n_colors = 216
epoch = None
exp_name = f'mmlm--n_colors={n_colors}c--mlm_probability=0.15'

checkpoint = load_checkpoint(exp_name, epoch=epoch)
print('Epoch:', checkpoint['epoch'])

Epoch: 999


In [4]:
!ls outputs/$exp_name

'epoch=149-step=22050.ckpt'  'epoch=649-step=95550.ckpt'
'epoch=199-step=29400.ckpt'  'epoch=699-step=102900.ckpt'
'epoch=19-step=2940.ckpt'    'epoch=69-step=10290.ckpt'
'epoch=249-step=36750.ckpt'  'epoch=749-step=110250.ckpt'
'epoch=299-step=44100.ckpt'  'epoch=799-step=117600.ckpt'
'epoch=29-step=4410.ckpt'    'epoch=79-step=11760.ckpt'
'epoch=349-step=51450.ckpt'  'epoch=849-step=124950.ckpt'
'epoch=399-step=58800.ckpt'  'epoch=899-step=132300.ckpt'
'epoch=39-step=5880.ckpt'    'epoch=89-step=13230.ckpt'
'epoch=449-step=66150.ckpt'  'epoch=949-step=139650.ckpt'
'epoch=499-step=73500.ckpt'  'epoch=999-step=147000.ckpt'
'epoch=49-step=7350.ckpt'    'epoch=99-step=14700.ckpt'
'epoch=549-step=80850.ckpt'  'epoch=9-step=1470.ckpt'
'epoch=599-step=88200.ckpt'   last.ckpt
'epoch=59-step=8820.ckpt'


In [5]:
config = load_config(exp_name)

# config.vocabulary_path = config.vocabulary_path.replace('/workspace/' ,'/workspace1/')
# config.base_path = config.base_path.replace('/workspace/' ,'/workspace1/')

Loading mmlm--n_colors=216c--mlm_probability=0.15 last checkpoint config from outputs/mmlm--n_colors=216c--mlm_probability=0.15/last.ckpt
Add new arg: permute_pixels = False
Add new arg: aug_zero_color = False


In [6]:
# pp.pprint(vars(config))

In [7]:
train_dataset, test_dataset, systematic_dataset, common_systematic_dataset = build_datasets(config)
config.pad_idx = train_dataset.pad_idx
config.n_tokens = train_dataset.n_tokens

complete_dataset = ConcatDataset((test_dataset, systematic_dataset))

In [8]:
# test_loaders = build_detailed_test_dataloaders(test_dataset, config) # type_of_tokens_to_test
# systematic_loaders = build_detailed_test_dataloaders(systematic_dataset, config) # type_of_tokens_to_test

In [9]:
model = MultimodalModel(config).to(device)
training_model = MultimodalPretrainingModel(model, config).to(device)
training_model.load_state_dict(checkpoint['state_dict'])

/home/fidelrio/.pyenv/versions/systematicity/lib/python3.7/site-packages/lightning/pytorch/utilities/parsing.py:270: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  f"Attribute {k!r} is an instance of `nn.Module` and is already saved during checkpointing."


<All keys matched successfully>

In [10]:
config.vocabulary_path, config.base_path

('/workspace1/fidelrio/CLEVR_CoGenT_v1.0/colored-v2/216c/vocab.txt',
 '/workspace1/fidelrio/CLEVR_CoGenT_v1.0/colored-v2/216c')

In [11]:
processor = test_dataset.processor
mask_token_idx = processor.vocabulary['[MASK]']
pad_token_idx = processor.vocabulary['[PAD]']

In [13]:
# collator = CollatorForMaskedLanguageModeling(config, processor)
collator = CollatorForMaskedSelectedTokens(config, processor, tokens=color_tokens)
# collator = CollatorForMaskedRandomSelectedTokens(config, processor, tokens=shapes_tokens, p=0.2)
# collator = IdentityCollator(config, processor)

In [16]:
trainer = Trainer(max_epochs=config.max_epochs,
                  accelerator="gpu",
                  devices=torch.cuda.device_count()
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
vocab = processor.vocabulary

relation_tokens = sorted(
    [vocab[w] for w in ['left', 'right', 'behind', 'front'] if w in vocab])
colors_tokens = sorted(
    [vocab[w] for w in ALL_POSSIBLE_COLORS if w in vocab])
#     [vocab[w] for w in ['blue', 'brown', 'cyan', 'green', 'red', 'purple', 'yellow', 'gray']])
shapes_tokens = sorted(
    [vocab[w] for w in ['cylinder', 'sphere', 'cube'] if w in vocab])
materials_tokens = sorted(
    [vocab[w] for w in ['metal', 'rubber'] if w in vocab])
sizes_tokens = sorted(
    [vocab[w] for w in ['small', 'large'] if w in vocab])

random_baseline = {
    'relation':  1 / len(relation_tokens),
    'color':  1 / len(color_tokens),
    'shapes':  1 / len(shapes_tokens),
    'materials':  1 / len(materials_tokens),
    'size':  1 / len(size_tokens),
    'identity':  1 / len(processor.vocabulary),
}

In [18]:
batch_size = 256
scenes_to_sample = 15_000

test_indices = random.sample(range(len(test_dataset)), k=scenes_to_sample)
pc_subset_test = Subset(test_dataset, test_indices)
systematic_indices = random.sample(range(len(systematic_dataset)), k=scenes_to_sample)
pc_subset_systematic = Subset(systematic_dataset, systematic_indices)
complete_indices = random.sample(range(len(complete_dataset)), k=scenes_to_sample)
pc_subset_complete = Subset(complete_dataset, complete_indices)


colors_collator = CollatorForMaskedSelectedTokens(config, processor, tokens=colors_tokens)
shapes_collator = CollatorForMaskedSelectedTokens(config, processor, tokens=shapes_tokens)
materials_collator = CollatorForMaskedSelectedTokens(config, processor, tokens=materials_tokens)
sizes_collator = CollatorForMaskedSelectedTokens(config, processor, tokens=sizes_tokens)
dlkwargs = {
    'batch_size': batch_size,
    'num_workers': int(os.environ.get("SLURM_CPUS_PER_TASK", 4)),
    'pin_memory': torch.cuda.is_available(),
}

test_loaders = {}
systematic_loaders = {}
complete_loaders = {}
for task, collator in [('colors', colors_collator),
                       ('shapes', shapes_collator),
                       ('materials', materials_collator),
                       ('sizes', sizes_collator)]:

    test_loaders[task] = DataLoader(
        pc_subset_test, shuffle=True, collate_fn=collator, **dlkwargs)
    systematic_loaders[task] = DataLoader(
        pc_subset_systematic, shuffle=True, collate_fn=collator, **dlkwargs)
    complete_loaders[task] = DataLoader(
        pc_subset_complete, shuffle=True, collate_fn=collator, **dlkwargs)
    

/home/fidelrio/.pyenv/versions/systematicity/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [19]:
feature_maps = []  # This will be a list of Tensors, each representing a feature map

def hook_feat_map(mod, inp, out):
    feature_maps.clear()
    feature_maps.append(out)

model.transformer.register_forward_hook(hook_feat_map)

In [20]:
# images.shape, scenes.shape, labels.shape

In [21]:
tasks = ['sizes', 'colors', 'materials', 'shapes']

In [22]:
def get_properties(scenes, indices=None):
    sizes = scenes[:,1:][:,0::5]
    colors = scenes[:,1:][:,1::5]
    materials = scenes[:,1:][:,2::5]
    shapes = scenes[:,1:][:,3::5]

    sizes = sizes[sizes != pad_token_idx].cpu()
    colors = colors[colors != pad_token_idx].cpu()
    materials = materials[materials != pad_token_idx].cpu()
    shapes = shapes[shapes != pad_token_idx].cpu()
    if indices is not None:
        sizes, colors, materials, shapes = sizes[indices], colors[indices], materials[indices], shapes[indices]
    return sizes, colors, materials, shapes

In [23]:
from collections import defaultdict
from tqdm import tqdm

feats_by_set = {}
gt_by_set = {}
props_by_set = {}
for test_name, loaders in [
        ('test', test_loaders), ('systematic', systematic_loaders), ('complete', complete_loaders)]:
    feats_by_task = defaultdict(list)
    gt_by_task = defaultdict(list)
    props_by_task = defaultdict(list)
    for task in tasks:
        for images, scenes, labels in tqdm(loaders[task]):
            images, scenes, labels = images.to(device), scenes.to(device), labels.to(device)
            cimages, cscenes, clabels = images, scenes, labels
            with torch.no_grad():
                output_logits = model(images, scenes)

                features = feature_maps[0]
                confidences = softmax(output_logits, dim=-1).max(dim=-1).values
                predictions = output_logits.argmax(dim=-1)

                scene_features = features.transpose(1,0)[:,-config.max_scene_size:]
                mask_idxs = (scenes == mask_token_idx)
                
                gts = labels[:,-config.max_scene_size:][mask_idxs].cpu()
                feats = scene_features[mask_idxs].cpu()
                props = get_properties(scenes)               
                
                gt_by_task[task].append(gts)
                feats_by_task[task].append(feats)
                props_by_task[task].append(props)
   
    feats_by_set[test_name] = feats_by_task
    gt_by_set[test_name] = gt_by_task
    props_by_set[test_name] = props_by_task

100%|██████████| 59/59 [00:50<00:00,  1.17it/s]


In [24]:
for test_name, feats_by_tasks in feats_by_set.items():
    for task_name, feats in feats_by_tasks.items():
        task_idx = tasks.index(task_name)
        feats_by_set[test_name][task_name] = torch.cat(feats)
        gt_by_set[test_name][task_name] = torch.cat(gt_by_set[test_name][task_name])    
        props_by_set[test_name][task_name] = torch.stack(
            [torch.cat(prop) for prop in zip(*props_by_set[test_name][task_name])], dim=-1)
        
        props_by_set[test_name][task_name][:,task_idx] = gt_by_set[test_name][task_name]

In [25]:
feats_by_set[test_name][task_name].shape, props_by_set[test_name][task_name].shape

(torch.Size([96807, 256]), torch.Size([96807, 4]))

In [26]:
objects_to_sample = 15_000

for test_name in feats_by_set:
    for task_name in feats_by_set[test_name]:
        object_indices = random.sample(range(len(feats_by_set[test_name][task_name])), k=objects_to_sample) 
        
        feats_by_set[test_name][task_name] = feats_by_set[test_name][task_name][object_indices]
        gt_by_set[test_name][task_name] = gt_by_set[test_name][task_name][object_indices]
        props_by_set[test_name][task_name] = props_by_set[test_name][task_name][object_indices]

In [27]:
feats_by_set[test_name][task_name].shape, props_by_set[test_name][task_name].shape

(torch.Size([15000, 256]), torch.Size([15000, 4]))

In [28]:
# torch.save({
#     'feats_by_set': feats_by_set,
#     'props_by_set': props_by_set,
# }, 'feats-tmp.pt')

# !ls -lah feats-tmp.pt
# !rm feats-tmp.pt

In [29]:
from torch import nn
from torch.optim import SGD
from sklearn.metrics import precision_recall_fscore_support

from collections import defaultdict
from torch.nn import functional as F
from tqdm.auto import tqdm
import copy


def create_linear_probe(n_features, hidden_size, n_targets):
    if isinstance(hidden_size, list):
        hidden_sizes = hidden_size
    else:
        hidden_sizes = [hidden_size]

    if n_targets == 2:
        n_targets = 1

    prev_hidden_size = hidden_sizes.pop(0)
    layers = [nn.Linear(n_features, prev_hidden_size)]
    for hidden_size in hidden_sizes:
        layers.append(nn.Linear(prev_hidden_size, hidden_size))
        prev_hidden_size = hidden_size
    layers.append(nn.Linear(prev_hidden_size, n_targets))
    linear_probe = nn.Sequential(*layers)
    
    for p in linear_probe:
        try:
            nn.init.kaiming_uniform_(p.weight)
        except AttributeError:
            pass

    linear_probe = linear_probe.to(device)
    return linear_probe
    
def train(linear_probe,
          optimizer,
          train_loader,
          val_loader,
          num_epochs,
          patience,
          n_targets):
    
    wait = 0
    best_model = None
    best_val_relevant_metric = 0

    is_binary_task = n_targets == 2
    criterion = F.binary_cross_entropy_with_logits if is_binary_task else F.cross_entropy
    
    linear_probe.train()

    metrics = defaultdict(list)
    for epoch in tqdm(range(num_epochs)):
        linear_probe.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            logit_pred = linear_probe(x).squeeze()
            y = y.float() if is_binary_task else y
            loss = criterion(logit_pred, y)
            loss.backward()
            optimizer.step()

            metrics['loss'].append(float(loss))
            if is_binary_task:
                pred = torch.sigmoid(logit_pred) > 0.5
                train_acc = (pred == y).sum() / y.shape[0]
                metrics['train_acc'].append(float(train_acc))
#                 prec, recall, f1 = calc_prec_recall_f1(pred, y)
#                 metrics['train_prec'].append(float(prec))
#                 metrics['train_recall'].append(float(recall))
#                 metrics['train_f1'].append(float(f1))
            else:
                pred = logit_pred.argmax(-1)
                train_acc = (pred == y).sum() / y.shape[0]
                metrics['train_acc'].append(float(train_acc))
                
                
        linear_probe.eval()
        cum_sum = 0
        cum_loss = 0
        cum_logit_pred, cum_y = [], []
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            bsz = x.shape[0]
            logit_pred = linear_probe(x).squeeze()
            y = y.float() if is_binary_task else y
            loss = criterion(logit_pred, y)
            cum_loss += float(loss) * bsz
            
            cum_logit_pred.append(logit_pred)
            cum_y.append(y)

        logit_pred = torch.cat(cum_logit_pred)
        y = torch.cat(cum_y)

        val_loss = cum_loss / len(val_dataset)
        metrics['val_loss'].append(float(val_loss))
        if is_binary_task:
            pred = torch.sigmoid(logit_pred) > 0.5
            val_acc = (pred == y).sum() / y.shape[0]
            metrics['val_acc'].append(float(val_acc))
#             prec, recall, f1 = calc_prec_recall_f1(pred, y)
#             metrics['val_prec'].append(float(prec))
#             metrics['val_recall'].append(float(recall))
#             metrics['val_f1'].append(float(f1))
        else:
            pred = logit_pred.argmax(-1)
            val_acc = (pred == y).sum() / y.shape[0]
            metrics['val_acc'].append(float(val_acc))

        # Early stopping
        val_relevant_metric = metrics['val_acc'][-1]
#         val_relevant_metric = metrics['val_f1'][-1] if is_binary_task else metrics['val_acc'][-1]
        if val_relevant_metric > best_val_relevant_metric:
            best_val_relevant_metric = val_relevant_metric
            best_model = copy.deepcopy(linear_probe)
            wait = 0
        else:
            wait += 1

        if wait > patience:
            break

    print(f'Finished in epoch {epoch}')
    return best_model, metrics, best_val_relevant_metric

def calc_prec_recall_f1(pred, y):
    p, r, f1, _ = precision_recall_fscore_support(
        y.cpu().detach().numpy(),
        pred.cpu().detach().numpy(),
        beta=1,
    )
    return p[1], r[1], f1[1]

In [30]:
tasks

['sizes', 'colors', 'materials', 'shapes']

In [45]:
test_name = 'complete'
feat_task_name = 'shapes'
gt_task_name = 'colors'

X = feats_by_set[test_name][feat_task_name]
y = props_by_set[test_name][feat_task_name][:,tasks.index(gt_task_name)]

tokens_to_class_map = {t: i for i, t in enumerate(set(y.tolist()))}
y = y.apply_(tokens_to_class_map.get) # transform to classes

X_train = X[:10_000,:].cuda()
X_val = X[10_000:12500,:].cuda()
X_test = X[12500:,:].cuda()
y_train = y[:10_000].cuda()
y_val = y[10_000:12500].cuda()
y_test = y[12500:].cuda()

In [46]:
feats_by_set[test_name][feat_task_name].shape, props_by_set[test_name][feat_task_name].shape

(torch.Size([15000, 256]), torch.Size([15000, 4]))

In [47]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

In [48]:
n_features = X.size(1)
n_targets = len(tokens_to_class_map)

n_features, n_targets

(256, 216)

In [52]:
from itertools import product

is_binary_task = n_targets == 2
relevant_metric_name = 'f1' if is_binary_task else 'acc'

all_metrics = {}
grid_best_relevant_metric = 0
grid_best_model = None
grid_best_metrics = None
best_hidden_size = None
best_lr = None
for exp_idx, (hidden_size, lr) in enumerate(product([32, 64, 128],
                                                    [1e-2, 1e-3, 1e-4]), start=1):

    print(f'Start experiment: {exp_idx}')

    linear_probe = create_linear_probe(n_features, [hidden_size, hidden_size], n_targets)
    optimizer = SGD(linear_probe.parameters(), lr=lr, momentum=0.9)

    best_model, metrics, best_relevant_metric = train(linear_probe,
                                                      optimizer,
                                                      train_loader,
                                                      val_loader,
                                                      num_epochs=500,
                                                      patience=20,
                                                      n_targets=n_targets,
                                                     )
    
    all_metrics[(hidden_size, lr)] = metrics
    if  best_relevant_metric > grid_best_relevant_metric:
        best_hidden_size = hidden_size
        best_lr = lr
        grid_best_model = best_model
        grid_best_metrics = metrics
        grid_best_relevant_metric = best_relevant_metric


    linear_probe = optimizer = None

    print(f'hidden_size={hidden_size} lr={lr}')
    print(f'Best val {relevant_metric_name}: ', float(best_relevant_metric))
    print('\n')

print('best_hidden_size:', best_hidden_size)
print('best_lr:', best_lr)

Start experiment: 1


  0%|          | 0/500 [00:00<?, ?it/s]

Finished in epoch 58
hidden_size=32 lr=0.01
Best val acc:  0.8163999915122986


Start experiment: 2


  0%|          | 0/500 [00:00<?, ?it/s]

Finished in epoch 352
hidden_size=32 lr=0.001
Best val acc:  0.7955999970436096


Start experiment: 3


...

Finished in epoch 94
hidden_size=128 lr=0.01
Best val acc:  0.8399999737739563


Start experiment: 8


  0%|          | 0/500 [00:00<?, ?it/s]

Finished in epoch 161
hidden_size=128 lr=0.001
Best val acc:  0.8187999725341797


Start experiment: 9


  0%|          | 0/500 [00:00<?, ?it/s]

Finished in epoch 499
hidden_size=128 lr=0.0001
Best val acc:  0.736799955368042


best_hidden_size: 128
best_lr: 0.01


In [53]:
# is_binary_task = len(set(group.values())) == 2
criterion = F.binary_cross_entropy_with_logits if is_binary_task else F.cross_entropy

grid_best_model.eval()
cum_loss = 0
cum_logit_pred = []
cum_y = []
for x, y in tqdm(test_loader):
    x, y = x.to(device), y.to(device)
    bsz = x.shape[0]
    logit_pred = grid_best_model(x).squeeze()
    y = y.float() if is_binary_task else y
    cum_logit_pred.append(logit_pred)
    cum_y.append(y)
    loss = criterion(logit_pred, y)
    cum_loss += float(loss) * bsz


logit_pred = torch.cat(cum_logit_pred)
y = torch.cat(cum_y)
grid_best_metrics['test_loss'] = (cum_loss / len(test_dataset))
if is_binary_task:
    pred = torch.sigmoid(logit_pred) > 0.5
    train_acc = (pred == y).sum() / y.shape[0]
    grid_best_metrics['test_acc'] = float(train_acc)
#     prec, recall, f1 = calc_prec_recall_f1(pred, y)
#     grid_best_metrics['test_prec'] = float(prec)
#     grid_best_metrics['test_recall'] = float(recall)
#     grid_best_metrics['test_f1'] = float(f1)
else:
    pred = logit_pred.argmax(-1)
    test_acc = (pred == y).sum() / y.shape[0]
    grid_best_metrics['test_acc'] = float(test_acc)

  0%|          | 0/10 [00:00<?, ?it/s]

In [54]:
print('test_acc: ', grid_best_metrics['test_acc'])

test_acc:  0.8283999562263489


In [56]:
grid_best_metrics.keys()

dict_keys(['loss', 'train_acc', 'val_loss', 'val_acc', 'test_loss', 'test_acc'])